# Load candidates

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
CACHE = '/home/anhphantq/vdb/cache_train'
import os
import pickle
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import psutil

train_data = []

with open(CACHE + '/metadata', 'rb') as f:
  metadata = pickle.load(f)

num_frags = min(metadata['num_frags'], 10000)
max_session = metadata['max_session']

with tqdm(total= int (psutil.virtual_memory().total  / 1024  / 1024 / 1024), desc='RAM (GB)', position=0, leave= False) as rambar:

  for i in tqdm(range(0, num_frags)):

    train_data.append(pd.read_parquet(CACHE + f'/data_{i}.parquet'))
    rambar.n= int(psutil.virtual_memory().used / 1024  / 1024 / 1024)
    rambar.refresh()




RAM (GB):   0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/1801 [00:00<?, ?it/s]

In [3]:
train_data = pd.concat(train_data)

In [4]:
candidates = train_data
candidates.shape

(98999726, 120)

In [5]:
import pandas as pd
pd.set_option('display.max_columns', candidates.shape[1])

In [6]:
candidates

,user,item,num_clicks,num_carts,num_orders,item_degree,item_pr,recent_num_clicks,recent_num_carts,recent_num_orders,item_recent_degree,item_recent_pr,recent_day7_type0,recent_day7_type1,recent_day7_type2,recent_day6_type0,recent_day6_type1,recent_day6_type2,recent_day5_type0,recent_day5_type1,recent_day5_type2,recent_day4_type0,recent_day4_type1,recent_day4_type2,recent_day3_type0,recent_day3_type1,recent_day3_type2,recent_day2_type0,recent_day2_type1,recent_day2_type2,recent_day1_type0,recent_day1_type1,recent_day1_type2,item_glob_last_action,item_glob_first_action,item_glob_time_decay_sum,item_glob_time_decay_sum_click,item_glob_time_decay_sum_cart,item_glob_time_decay_sum_order,item_glob_item_glob_durability,qou_num_cosub_mean,qou_coclick_sub_coef_mean,qou_cocart_sub_coef_mean,qou_coorder_sub_coef_mean,qou_coclick_time_decay_mean,qou_cocart_time_decay_mean,qou_coorder_time_decay_mean,qou_num_appearance_mean,qou_num_in_k_most_recent_items_mean,qou_num_happend_later_mean,qou_num_happend_before_mean,qou_happend_later_ratio_mean,qou_last_interact_mean,qou_lincom_sub_coef_1_3_10_mean,qou_lincom_time_decay_1_3_10_mean,qou_lincom_sub_coef_1_10_3_mean,qou_lincom_time_decay_1_10_3_mean,qou_lincom_sub_coef_3_1_10_mean,qou_lincom_time_decay_3_1_10_mean,qou_lincom_sub_coef_3_10_1_mean,qou_lincom_time_decay_3_10_1_mean,qou_lincom_sub_coef_10_1_3_mean,qou_lincom_time_decay_10_1_3_mean,qou_lincom_sub_coef_10_3_1_mean,qou_lincom_time_decay_10_3_1_mean,qou_lincom_sub_coef_0.5_10_0.5_mean,qou_lincom_time_decay_0.5_10_0.5_mean,qou_lincom_sub_coef_10_0.5_0.5_mean,qou_lincom_time_decay_10_0.5_0.5_mean,qou_lincom_sub_coef_0.5_0.5_10_mean,qou_lincom_time_decay_0.5_0.5_10_mean,qou_num_cosub_sqrt_num_cousers,qou_coclick_sub_coef_sqrt_num_cousers,qou_cocart_sub_coef_sqrt_num_cousers,qou_coorder_sub_coef_sqrt_num_cousers,qou_coclick_time_decay_sqrt_num_cousers,qou_cocart_time_decay_sqrt_num_cousers,qou_coorder_time_decay_sqrt_num_cousers,qou_num_appearance_sqrt_num_cousers,qou_num_in_k_most_recent_items_sqrt_num_cousers,qou_num_happend_later_sqrt_num_cousers,qou_num_happend_before_sqrt_num_cousers,qou_happend_later_ratio_sqrt_num_cousers,qou_last_interact_sqrt_num_cousers,num_sub,consistency,num_actions,user_degree,user_pr,user_recent_degree,user_recent_pr,fitness,inter_clicks,inter_carts,inter_orders,inter_num_sub,inter_time_decay,inter_lts,inter_fts,inter_durability,inter_num_interacts,is_level1,user_clicks_cnt,user_carts_cnt,user_orders_cnt,user_total_actions_cnt,user_clicks_ratio,user_carts_ratio,user_orders_ratio,user_ss_ts_max,user_ss_ts_min,user_ss_ts_mean,item_item_cnt,item_item_clicks_cnt,item_item_carts_cnt,item_item_orders_cnt,item_item_total_actions_cnt,item_item_clicks_ratio,item_item_carts_ratio,item_item_orders_ratio
0,11789523.0,12911609.0,25173.0,3023.0,1034.0,17394.0,6.987022e-05,8302.0,930.0,312.0,5410.0,4.944853e-05,658.0,29.0,8.0,1024.0,129.0,39.0,1289.0,163.0,56.0,1108.0,145.0,44.0,1047.0,121.0,51.0,977.0,109.0,41.0,984.0,97.0,30.0,20.999699,0.005752,11791.661133,10165.686523,1237.395752,388.578766,20.993946,0.550737,0.491885,0.349321,0.000000,0.274235,0.397164,0.000000,0.668458,0.446845,0.559117,0.858712,0.458966,1.019011,0.395255,0.244055,0.420576,0.311150,0.455799,0.255678,0.465071,0.295658,0.487778,0.273009,0.488271,0.276097,0.409677,0.342639,0.490908,0.274317,0.340369,0.190806,0.036116,0.002254,0.000097,0.000000,0.000524,5.372495e-05,0.000000e+00,1.138420,0.001570,0.017273,0.039256,0.009663,56958.121094,0.0,1.000000,1.0,1.0,1.344081e-08,1.0,2.229009e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.000000,0.000000,0.0,1.661348e+09,1.661348e+09,1.661348e+09,32546.0,27910.0,3555.0,1081.0,32546.0,0.857555,0.109230,0.033215
1,11789523.0,12927359.0,65.0,11.0,7.0,48.0,2.086333e-07,34.0,5.0,3.0,21.0,2.298855e-07,5.0,1.0,1.0,7.0,3.0,1.0,3.0,0.0,0.0,7.0,1.0,0.0,2.0,1.0,0.0,4.0,0.0,0.0,6.0,1.0,1.0,20.947731,0.284745,48.843647,38.231949,7.489288,3.122410,20.662987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

# Train params

In [7]:
from meow_utils import *
from catboost.utils import eval_metric

PARAMS = {
    'learning_rate': 0.5,
    'iterations': 1000,
    'early_stopping_rounds' : 20,
    'max_bin': 256,
    'depth': 7,
    'eval_metric': 'RecallAt:top=20;border=0.5',
    'use_best_model': True,
    'thread_count': -1,
}
N_SPLITS = 5
NUM_SAMPLES = 3
MODEL_PATH = '/home/anhphantq/meowmeow'
VERSION = '9'

# Train for each type

In [8]:
from meow_utils import get_len_group

In [9]:
import pandas as pd 
import numpy as np
tar = pd.read_parquet('/home/anhphantq/otto/splitted_data/test_labels.parquet')
tar = tar.loc[(tar['session'] <= candidates['user'].max()) & (tar['session'] >= candidates['user'].min()) ]
aids = tar.ground_truth.explode().astype('int32').rename('item')
tar = tar[['session', 'type']].rename({'session':'user'},axis=1)
tar = tar.merge(aids, left_index=True, right_index=True, how='left')
tar['type'] = tar['type'].apply(lambda x: 0 if x == 'clicks' else 1 if x == 'carts' else 2)
tar['label'] = 1
tar['item'] = tar['item'] + max_session
tar['user_type'] = tar['user'] * 10 + tar['type']

In [10]:
labels = []
users = candidates.user.values
items = candidates.item.values
from tqdm.notebook import tqdm
for i in tqdm(range(3)):
  tar_small = tar[tar['type'] == i].sort_values(['user', 'item']).set_index(['user', 'item'])
  
  tar_small_index = set(tar_small.index.tolist())
  label = []

  for i in tqdm(range(candidates.shape[0]), leave = False):
    
    label.append(1 if (users[i], items[i]) in tar_small_index else 0)
  
  labels.append(label)

del tar, tar_small_index
  



  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/98999726 [00:00<?, ?it/s]

  0%|          | 0/98999726 [00:00<?, ?it/s]

  0%|          | 0/98999726 [00:00<?, ?it/s]

In [11]:
labels = np.array(labels)

In [12]:
from sklearn.model_selection import GroupKFold
import os
import matplotlib.pyplot as plt
import shutil
from meow_utils import plot_importance
from catboost import CatBoostRanker, Pool
import seaborn as sns
sns.set()
%matplotlib inline
def train(t, id_type, draw_importance = False):
  global candidates, labels
  name = f'/{t}_only_'
  try:
    shutil.rmtree(MODEL_PATH + name + VERSION)
  except:
    pass
  else:
    print('remove old version')
  fig, axs = plt.subplots(1, 3, figsize = (300, 30))
  skf = GroupKFold(n_splits=N_SPLITS)
  
  
  X_all = candidates.iloc[:, 2 :]
  y_all = labels[id_type]
  print('Building pool...')
  data_all = Pool(X_all, y_all, thread_count = os.cpu_count())
  print('Pool builed!!!!!!')
  for fold,(train_idx, valid_idx) in enumerate(skf.split(candidates, labels[id_type], groups=candidates['user'] )):
      neg_index = np.ravel(np.argwhere(labels[id_type][train_idx] == 0))
      pos_index = np.ravel(np.argwhere(labels[id_type][train_idx] == 1))
      #downsample many times
      for sample in range(NUM_SAMPLES):
        print(f'FOLD: {fold} Sample: {sample}')
        n_index = np.random.permutation(neg_index)[:len(pos_index) * 20]
        index = np.union1d(n_index, pos_index)
        
        data_train = data_all.slice(train_idx).slice(index)
        data_train.set_group_id(candidates['user'].values.astype(int)[train_idx][index])
        data_valid = data_all.slice(valid_idx)
        data_valid.set_group_id(candidates['user'].values.astype(int)[valid_idx])

        params = {'thread_count' : os.cpu_count(),
        **PARAMS}
        model = CatBoostRanker(**params)
        
        model.fit(data_train, eval_set = data_valid)
        
        try:
          os.mkdir(MODEL_PATH + name + VERSION)
        except:
          pass
        if fold < 5 and sample == 0 and draw_importance:
          plot_importance(X_all.columns.tolist(), model.get_feature_importance(data = data_valid, type = 'LossFunctionChange'), ax = axs[fold])
          axs[fold].bar_label(axs[fold].containers[0])
        model.save_model(MODEL_PATH + name + VERSION + f'/meowmewo_fold{fold}_sample{sample}')
  


In [13]:
for id_type, t in enumerate(['clicks', 'carts', 'orders']):
  train(t, id_type)

Building pool...
Pool builed!!!!!!
FOLD: 0 Sample: 0
0:	learn: 0.9999147	test: 0.8966000	best: 0.8966000 (0)	total: 2.96s	remaining: 49m 14s
1:	learn: 0.9999504	test: 0.9075447	best: 0.9075447 (1)	total: 5.28s	remaining: 43m 56s
2:	learn: 0.9999663	test: 0.9112683	best: 0.9112683 (2)	total: 7.91s	remaining: 43m 48s
3:	learn: 0.9999742	test: 0.9156264	best: 0.9156264 (3)	total: 10.5s	remaining: 43m 41s
4:	learn: 0.9999772	test: 0.9162252	best: 0.9162252 (4)	total: 12.9s	remaining: 42m 43s
5:	learn: 0.9999752	test: 0.9179105	best: 0.9179105 (5)	total: 15.2s	remaining: 41m 58s
6:	learn: 0.9999772	test: 0.9181921	best: 0.9181921 (6)	total: 17.7s	remaining: 41m 45s
7:	learn: 0.9999811	test: 0.9191279	best: 0.9191279 (7)	total: 19.9s	remaining: 41m 5s
8:	learn: 0.9999792	test: 0.9198219	best: 0.9198219 (8)	total: 22s	remaining: 40m 21s
9:	learn: 0.9999802	test: 0.9202779	best: 0.9202779 (9)	total: 24.1s	remaining: 39m 43s
10:	learn: 0.9999802	test: 0.9206665	best: 0.9206665 (10)	total: 26.6s